In [1]:
# Kullanacağım Kütüphaneleri projeme import ediyorum
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import Adam

In [ ]:
# Veri seti yolunu alıyorum
dataset_path = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

# ImageDataGenerator ile veri yükleme 
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),  # Görüntü boyutu 64x64 piksel olarak ayarlanıyor
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
# ANN modelini kur
model = Sequential()

# Giriş katmanı - Görüntüleri 64x64x3 boyutundan düzleştir 
model.add(Flatten(input_shape=(64, 64, 3)))

# Gizli katmanlar
model.add(Dense(128, activation='relu'))  # 128 nöronlu gizli katman
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))   # 64 nöronlu gizli katman
model.add(Dropout(0.2))

# Çıkış katmanı - Balık türlerini sınıflandırmak için
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Modeli derle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modelin özetini görüntüle
model.summary()

In [ ]:
# Modeli eğit
history = model.fit(train_generator, validation_data=validation_generator, epochs=80) # deneme yanılma yoluyla ortalama 80 epoch un benim için iyi olduğuna karar verdim.


In [ ]:
# Test veri setinde modelin başarısını değerlendirme
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test doğruluğu: {test_acc:.4f}")


In [ ]:
# Eğitim doğruluğu ve kaybını görselleştir
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend(loc='upper right')
plt.show()


In [ ]:
# İlk 5 görüntü için tahmin yapalım
predictions = model.predict(validation_generator)

# İlk 5 tahmini ve gerçek etiketleri karşılaştıralım
for i in range(5):
    img, label = validation_generator[i]
    plt.imshow(img[0])
    plt.title(f"Tahmin: {np.argmax(predictions[i])}, Gerçek: {np.argmax(label[0])}")
    plt.show()
